In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import time
import random

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from acquire import *

### All Season & Robot Links

In [2]:
# loading the JSON file with all season & robot links
robot_links_dict = load_from_json("all_season_robot_links.json")

In [3]:
total_links = sum(len(links) for links in robot_links_dict.values())

In [4]:
print(total_links)

394


In [ ]:
# Load the current index from file, or start from the beginning
try:
    with open('current_index.txt', 'r') as f:
        current_index = int(f.read())
except:
    current_index = 0

# Iterate through the dictionary of links starting from the current index
for i, (season, links) in enumerate(robot_links_dict.items()):
    if i < current_index:
        continue
    for link in links:
        # Extract robot name from the link
        robot_name = link.split('/')[-2]

        # Send link to scraper browser
        driver.get(link)

        # Get rid of cookies_popup
        #get_rid_of_cookies_popup(driver)

        # Get table elements
        div_element = driver.find_element_by_css_selector(".grid-1-1.last")
        table_elements = div_element.find_elements_by_css_selector("table")

        # Check if there are two tables
        if len(table_elements) == 2:
            stats_history_html = table_elements[0].get_attribute("outerHTML")
            match_history_html = table_elements[1].get_attribute("outerHTML")

            # Create dataframes from the tables
            stats_df = pd.read_html(stats_history_html)[0]
            match_history_df = pd.read_html(match_history_html)[0]

            # Save the dataframes to file
            stats_df.to_pickle(f'{robot_name}_stats.pkl')
            match_history_df.to_pickle(f'{robot_name}_match_history.pkl')
        
        # Sleep for a random amount of time between 3 and 7 seconds
        time.sleep(random.uniform(1, 5))
        
    # Increment the current index and save to file
    current_index += 1
    with open('current_index.txt', 'w') as f:
        f.write(str(current_index))
        
# Close the driver
driver.quit()

In [ ]:
df = pd.read_parquet('robot_data_captain-shredderator-wcvii.parquet')

In [ ]:
# Read nested dataframe from Parquet file
pf = fp.ParquetFile('robot_data.parquet')
df = pf.to_pandas()

In [ ]:
# Access the 'stats' dataframe for robot 'robot1'
stats_df = df.loc['robot1', 'stats']